<a href="https://colab.research.google.com/github/ZZANZU/DeeeeeepLearning/blob/master/tflite_freeze_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Get Trained Model & Graph & Checkpoint

In [0]:
from google.colab import files # mounting google drive

import tensorflow as tf
import numpy as np

In [0]:
W = tf.Variable(initial_value=tf.random_normal([1]), name='weight', trainable=True)
b = tf.Variable(initial_value=0.001, name='bias', trainable=True)

x = tf.placeholder(dtype=tf.float32, shape=[1], name='x')
y = tf.add(tf.multiply(W, x), b, name='output')

In [0]:
init = tf.global_variables_initializer()

In [0]:
saver = tf.train.Saver()
save_path = "data/"
model_save = save_path + "model.ckpt"

In [0]:
with tf.Session() as sess:
    sess.run(init)
    op = sess.run(y, feed_dict={x: np.reshape(1.5, [1])})
    saver.save(sess, model_save)
    tf.train.write_graph(sess.graph_def, save_path, 'savegraph.pbtxt')

## Download

https://stackoverflow.com/questions/48678033/how-do-i-save-my-trained-neural-net

In [10]:
import os
print(os.getcwd())
print(os.listdir('data'))

/content
['model.ckpt.data-00000-of-00001', 'model.ckpt.meta', 'model.ckpt.index', 'savegraph.pbtxt', 'checkpoint']


In [0]:
files.download("data/savegraph.pbtxt")

In [0]:
files.download("data/model.ckpt.meta")

# 2. Freeze a TensorFlow Model

In [0]:
from tensorflow.python.tools import freeze_graph

## Required files
1) Saved model
2) Saved model's graph

In [0]:
# Freeze the graph
save_path = "data/"
MODEL_NAME = 'Sample_model'
input_graph_path = save_path + 'savegraph.pbtxt'
checkpoint_path = save_path + 'model.ckpt'
input_saver_def_path = ""
input_binary = False
output_node_names = "output"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = save_path + 'frozen_model_' + MODEL_NAME + '.pb'
clear_devices = True

In [20]:
freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                         input_binary, checkpoint_path, output_node_names,
                         restore_op_name, filename_tensor_name,
                         output_frozen_graph_name, clear_devices, "")

Instructions for updating:
Use tf.gfile.GFile.
INFO:tensorflow:Restoring parameters from data/model.ckpt
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


# 3. Import the frozen model and define input and output nodes

In [0]:
graph_def_file = 'data/frozen_model_Sample_model.pb' # our pb file

In [0]:
input_arrays = ['x'] # input node, 내가 그래프 만들 때 사용한 input의 이름으로 설정해야됨. output도 동일!
output_arrays = ['output'] # output node

https://www.google.co.kr/search?q=tf.contrib.lite.TocoConverter.from_frozen_graph&oq=tf.contrib&aqs=chrome.0.69i59j69i57j69i61.2330j0j9&sourceid=chrome&ie=UTF-8

In [0]:
# DEPRECATED : tf.contrib.lite.TocoConverter.from_frozen_graph
converter = tf.contrib.lite.TFLiteConverter.from_frozen_graph(graph_def_file, input_arrays, output_arrays)

In [66]:
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

688